In [1]:
import pandas as pd
import requests
import unittest
import os
import shutil

### clean folder structure

In [2]:
if os.path.exists("db"):
    shutil.rmtree("db")

### start demandForecastManager

In [3]:
!docker-compose down
!docker build -t demandforecastmanager .
!docker-compose up -d

[+] Running 0/0
 ⠋ Container demandforecastmanager-dfm-1  Stopping                         0.1s 
[+] Running 0/1
 ⠙ Container demandforecastmanager-dfm-1  Stopping                         0.2s 
[+] Running 2/1
 ✔ Container demandforecastmanager-dfm-1  Removed                          0.2s 
 ✔ Network demandforecastmanager_default  Removed                          0.0s 
[+] Building 0.0s (0/1)                                    docker:desktop-linux
[+] Building 0.0s (11/11) FINISHED                         docker:desktop-linux
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 321B                                       0.0s
 => [internal] load metadata for docker.io/library/python:3.11             0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [1/6] FROM docker.io/library/python:3.11                        

### run unit tests

In [4]:
class testDemandForecastManager(unittest.TestCase):
    base_url = "http://0.0.0.0:5011/"
    def test_0_index(self):
        response = requests.get(url=self.base_url)
        self.assertEqual(response.status_code, 200)
        self.assertEqual(response.text, '{"message":"Index of demandForecastManager."}\n')

    def test_1_get_health(self):
        response = requests.get(url=self.base_url + "getHealth")
        self.assertEqual(response.status_code, 200)

    def test_3_get_ids(self):
        response = requests.get(url=self.base_url + "getIds")
        self.assertEqual(response.status_code, 200)
        self.assertEqual(response.text, '{"buildingIds":{}}\n')

    def test_4_add_ids(self):
        headers = {
            "buildingIdentifier": "Building_Test",
            "smartmeterIdentifiers": 'Smartmeter_3,Smartmeter_4'
        }
        response = requests.post(url=self.base_url + "addId", headers=headers)
        self.assertEqual(response.status_code, 200)
        self.assertEqual(response.text, '{"message":"Created Database entry for Building_Test"}\n')
    
    def test_5_get_ids(self):
        response = requests.get(url=self.base_url + "getIds")
        self.assertEqual(response.status_code, 200)
        self.assertEqual(response.text, '{"buildingIds":{"Building_Test":["Smartmeter_3","Smartmeter_4"]}}\n')

    def test_6_update_ids(self):
        headers = {
            'buildingIdentifier': 'Building_Test',
            'smartmeterIdentifiers': 'Smartmeter_4,Smartmeter_5,Smartmeter_6'
        }
        response = requests.put(url=self.base_url + "updateId", headers=headers)
        self.assertEqual(response.status_code, 200)
        self.assertEqual(response.text, '{"message":"Updated Smartmeter Ids for Building_Test"}\n')

    def test_7_get_ids(self):
        response = requests.get(url=self.base_url + "getIds")
        self.assertEqual(response.status_code, 200)
        self.assertEqual(response.text, '{"buildingIds":{"Building_Test":["Smartmeter_3","Smartmeter_4","Smartmeter_5","Smartmeter_6"]}}\n')

    def test_8_delete_smartmeter_ids(self):
        headers = {
            'buildingIdentifier': "Building_Test",
            'smartmeterIdentifiers': 'Smartmeter_3,Smartmeter_4'
        }
        
        response = requests.delete(url=self.base_url + "deleteSmartmeterId", headers=headers)
        self.assertEqual(response.status_code, 200)
        self.assertEqual(response.text, '{"message":"Smartmeter Identifiers successfully removed from database."}\n')

    def test_91_get_ids(self):
        response = requests.get(url=self.base_url + "getIds")
        self.assertEqual(response.status_code, 200)
        self.assertEqual(response.text, '{"buildingIds":{"Building_Test":["Smartmeter_5","Smartmeter_6"]}}\n')

    def test_99_delete_building_id(self):
        headers = {
            'buildingIdentifier': 'Building_Test'
        }
        response = requests.delete(url=self.base_url + "deleteBuildingId", headers=headers)
        self.assertEqual(response.status_code, 200)
        self.assertEqual(response.text, '{"message":"Building identifier successfully removed from database."}\n')

In [5]:
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

..........
----------------------------------------------------------------------
Ran 10 tests in 0.315s

OK


### add data to database

In [ ]:
## add new builidng id

## add new smartmeter id

## upload data for building

## retrieve data for building